In [1]:
%load_ext Cython

In [2]:
from math import sqrt
from array import array

def A(i, j):
    """i: row, j: column, ZERO-based."""
    return 1.0 / (((i + j) * (i + j + 1) >> 1) + i + 1)

def A_times_u(u, v):
    """v = Au, A: nxn Matrix, u: n vector."""
    u_len = len(u)
    
    for i in range(u_len):
        partial_sum = 0.0
        for j in range(u_len):
            partial_sum += A(i, j) * u[j]
            
        v[i] = partial_sum
        
def At_times_u(u, v):
    """v = A^T u, A: nxn Matrix, u: n vector."""
    u_len = len(u)
    
    for i in range(u_len):
        partial_sum = 0.0
        for j in range(u_len):
            partial_sum += A(j, i) * u[j]
            
        v[i] = partial_sum
        
def B_times_u(u, out, tmp):
    """Bu = A^T A u, tmp = A u, out = A^T tmp."""
    A_times_u(u, tmp)
    At_times_u(tmp, out)
    
def spectral_norm(n):
    u = array("d", [1.0] * n)
    v = array("d", [0.0] * n)
    tmp = array("d", [0.0] * n)
    
    # B^n u, set n=20 to adapt u to 
    # closely aligned with the principal eigenvector.
    for _ in range(10):
        B_times_u(u, v, tmp)
        B_times_u(v, u, tmp)
        
    vBv, vv = 0.0, 0.0
    
    for ue, ve in zip(u, v):
        vBv += ue * ve
        vv += ve * ve
    
    return sqrt(vBv / vv)

def main(n):
    print("%0.9f" % spectral_norm(n))

In [3]:
import cProfile as pf

In [4]:
pf.run('main(400)', sort='time')

1.274224081
         6400138 function calls in 3.858 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  6400000    2.465    0.000    2.465    0.000 <ipython-input-2-f499672ca18f>:4(A)
       20    0.707    0.035    1.950    0.098 <ipython-input-2-f499672ca18f>:8(A_times_u)
       20    0.685    0.034    1.907    0.095 <ipython-input-2-f499672ca18f>:19(At_times_u)
        1    0.000    0.000    3.858    3.858 <ipython-input-2-f499672ca18f>:35(spectral_norm)
       20    0.000    0.000    3.858    0.193 <ipython-input-2-f499672ca18f>:30(B_times_u)
        3    0.000    0.000    0.000    0.000 iostream.py:180(schedule)
       40    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    3.858    3.858 {built-in method builtins.exec}
        1    0.000    0.000    3.858    3.858 <ipython-input-2-f499672ca18f>:54(main)
        2    0.000    0.000    0.000    0.000 iostream.py:342(write)
    

In [5]:
%%cython
from math import sqrt
from array import array

def A0(i, j):
    """i: row, j: column, ZERO-based."""
    return 1.0 / (((i + j) * (i + j + 1) >> 1) + i + 1)

def A_times_u0(u, v):
    """v = Au, A: nxn Matrix, u: n vector."""
    u_len = len(u)
    
    for i in range(u_len):
        partial_sum = 0.0
        for j in range(u_len):
            partial_sum += A0(i, j) * u[j]
            
        v[i] = partial_sum
        
def At_times_u0(u, v):
    """v = A^T u, A: nxn Matrix, u: n vector."""
    u_len = len(u)
    
    for i in range(u_len):
        partial_sum = 0.0
        for j in range(u_len):
            partial_sum += A0(j, i) * u[j]
            
        v[i] = partial_sum
        
def B_times_u0(u, out, tmp):
    """Bu = A^T A u, tmp = A u, out = A^T tmp."""
    A_times_u0(u, tmp)
    At_times_u0(tmp, out)
    
def spectral_norm0(n):
    u = array("d", [1.0] * n)
    v = array("d", [0.0] * n)
    tmp = array("d", [0.0] * n)
    
    # B^n u, set n=20 to adapt u to 
    # closely aligned with the principal eigenvector.
    for _ in range(10):
        B_times_u0(u, v, tmp)
        B_times_u0(v, u, tmp)
        
    vBv, vv = 0.0, 0.0
    
    for ue, ve in zip(u, v):
        vBv += ue * ve
        vv += ve * ve
    
    return sqrt(vBv / vv)

def main0(n):
    print("%0.9f" % spectral_norm0(n))

In [8]:
pf.run("main0(400)", sort='time')

1.274224081
         35 function calls in 1.713 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.711    1.711    1.713    1.713 {_cython_magic_f41508800d19f149a51a027ed7e2c3da.main0}
        3    0.002    0.001    0.002    0.001 iostream.py:180(schedule)
        2    0.000    0.000    0.003    0.001 iostream.py:342(write)
        3    0.000    0.000    0.000    0.000 {built-in method posix.urandom}
        1    0.000    0.000    1.713    1.713 {built-in method builtins.exec}
        3    0.000    0.000    0.000    0.000 threading.py:1060(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1102(is_alive)
        2    0.000    0.000    0.000    0.000 iostream.py:284(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:297(_schedule_flush)
        3    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
        3    0.000    0.000   

In [30]:
%%cython -a
# cython: cdivison=True, boundscheck=False, wrapround=False

from libc.math cimport sqrt
import numpy as np
#from cpython.array cimport array

cdef inline double A1(int i,int j):
    """i: row, j: column, ZERO-based."""
    return 1.0 / (((i + j) * (i + j + 1) >> 1) + i + 1)

cdef void A_times_u1(double[::1] u, double[::1] v):
    """v = Au, A: nxn Matrix, u: n vector."""
    cdef:
        int u_len = u.shape[0]
        int i
        double partial_sum
    
    for i in range(u_len):
        partial_sum = 0.0
        for j in range(u_len):
            partial_sum += A1(i, j) * u[j]
            
        v[i] = partial_sum
        
cdef void At_times_u1(double[::1] u, double[::1] v):
    """v = A^T u, A: nxn Matrix, u: n vector."""
    cdef:
        int u_len = u.shape[0]
        int i
        double partial_sum
    
    for i in range(u_len):
        partial_sum = 0.0
        for j in range(u_len):
            partial_sum += A1(j, i) * u[j]
            
        v[i] = partial_sum
        
cdef inline void B_times_u1(double[::1] u, double[::1] out, double[::1] tmp):
    """Bu = A^T A u, tmp = A u, out = A^T tmp."""
    A_times_u1(u, tmp)
    At_times_u1(tmp, out)
    
cdef double spectral_norm1(int n):
    cdef:
        double[::1] u = np.ones((n,), dtype=np.double)
        double[::1] v = np.empty((n,), dtype=np.double)
        double[::1] tmp = np.empty((n,), dtype=np.double)
        #double[::1] u = array('d', [1.0]*n)
        #double[::1] v = array('d', [0.0]*n)
        #double[::1] tmp = array('d', [0.0]*n)
        int i, j
        double vBv=0.0, vv=0.0#, ue, ve
    
    # B^n u, set n=20 to adapt u to 
    # closely aligned with the principal eigenvector.
    for i in range(10):
        B_times_u1(u, v, tmp)
        B_times_u1(v, u, tmp)
        
    #for ue, ve in zip(u, v):
    #    vBv += ue * ve
    #    vv += ve * ve
    for j in range(n):
        vBv += u[j] * v[j]
        vv += v[j] * v[j]
    
    return sqrt(vBv / vv)

cpdef void main1(n):
    print("%0.9f" % spectral_norm1(n))

In [40]:
pf.run("main1(400)", sort='time')

1.274224081
         38 function calls in 0.012 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.010    0.010    0.012    0.012 {built-in method _cython_magic_11ff13a900cc2fede1824e43f31ba3b4.main1}
        3    0.001    0.000    0.001    0.000 {built-in method posix.urandom}
        2    0.000    0.000    0.002    0.001 iostream.py:342(write)
        3    0.000    0.000    0.002    0.001 iostream.py:180(schedule)
        1    0.000    0.000    0.012    0.012 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 numeric.py:150(ones)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.copyto}
        3    0.000    0.000    0.000    0.000 threading.py:1102(is_alive)
        3    0.000    0.000    0.000    0.000 iostream.py:87(_event_pipe)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.empty}
        2    0.000    0.000

In [34]:
pf.run("main1(5500)", sort='time')

1.274224153
         38 function calls in 1.940 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.939    1.939    1.940    1.940 {built-in method _cython_magic_11ff13a900cc2fede1824e43f31ba3b4.main1}
        2    0.000    0.000    0.000    0.000 iostream.py:342(write)
        3    0.000    0.000    0.000    0.000 iostream.py:180(schedule)
        1    0.000    0.000    1.940    1.940 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.copyto}
        3    0.000    0.000    0.000    0.000 {built-in method posix.urandom}
        2    0.000    0.000    0.000    0.000 iostream.py:284(_is_master_process)
        1    0.000    0.000    0.000    0.000 numeric.py:150(ones)
        3    0.000    0.000    0.000    0.000 threading.py:1102(is_alive)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.empty}
        1    0.000 